In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor,BitsAndBytesConfig
from PIL import Image
import requests
from io import BytesIO

In [ ]:
!pip install torch -q

In [ ]:
!pip install transformers pillow requests bitsandbytes accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.8 MB/s eta 0:00:00


In [ ]:
model_name = "Qwen/Qwen2-VL-2B-Instruct"

model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="sdpa"
)

processor = AutoProcessor.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 772fb556-cf34-4ece-affb-60e98accddd0)')' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen2-VL-2B-Instruct/resolve/main/model.safetensors
Retrying in 1s [Retry 1/5].


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [ ]:
!pip install hf_xet -q

In [ ]:
question = "Bana 500 kalorilik bir menü önerirmisin?"

text_prompt = processor.apply_chat_template(
    [{"role": "user", "content": question}],
    tokenize=False,
    add_generation_prompt=True
)

inputs = processor(
    text=[text_prompt],
    images=None,
    return_tensors="pt",
    padding=True
).to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=250,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

answer = processor.batch_decode(output, skip_special_tokens=True)[0]
print(answer.split("assistant")[-1].strip())


Tabii ki, bu menüdeki kalori sayısı 500'dir:

1. 2 adet kahve: 100 kalori
2. 1 tane tohumlu kek: 100 kalori
3. 1 adet süt: 40 kalori
4. 1 adet çay: 60 kalori
5. 1 adet biber: 70 kalori
6. 1 adet mısır: 80 kalori
7. 1 adet zeytinyağı: 90 kalori
8. 1 adet kuru balık: 100 kalori
9. 1 adet kuru sebzeler: 100 kalori
10. 1 adet kuru domates: 100 kalori

Bu menüdeki toplam kalori sayısı 500 kalori olacaktır. Bu menüdeki besinler arasında çeşitli besinler bulunur ve bu besinlerin kalori açısından farklılık gösterir. Bu nedenle, her kişi için belirli bes


In [ ]:
import requests
from PIL import Image
from io import BytesIO

# 1. Resmi Al
url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQTqYqldSyrIb2y5m_oCLy4VYsL6meCvP2jXg&s"
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
image = Image.open(BytesIO(response.content))

# 2. PROMPT'U (Modeli base model için kırmanın yolu)
prompt = """
Bu bir görsel analiz ve nesne tanıma testidir.
Fotoğraftaki yiyecekleri tespit et (Hamburger, patates vb.).
Her bir yiyecek için genel besin veritabanlarında (örneğin USDA) kabul edilen ortalama kalori değerlerini yaz.
Kişisel tavsiye verme, sadece nesneleri ve standart kalori değerlerini listele.
Sonuçları şu formatta ver:
- Yiyecek Adı: Tahmini Kalori
- Toplam: Tahmini Toplam
"""

conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": prompt},
        ],
    }
]

# 3. İşleme
text_prompt = processor.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
inputs = processor(text=[text_prompt], images=[image], return_tensors="pt", padding=True).to("cuda")

# 4. Cevabı Al
output = model.generate(**inputs, max_new_tokens=256)
print(processor.batch_decode(output, skip_special_tokens=True)[0].split("assistant")[-1])


Bu fotoğraf, McDonald's'ın "McGriddle" nesnesini gösteriyor. Bu yiyecek, 100 gram (3.5 un) bir kahvaltı için 300 kalori içerir. Bu, genel besin veritabanlarında kabul edilen ortalamalı kalori değerlerinin 100 gram (3.5 un) bir kahvaltı için 300 kalori olduğunu gösterir.


In [ ]:
!pip install torch transformers peft datasets bitsandbytes accelerate qwen-vl-utils -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 17.0 MB/s eta 0:00:00


In [ ]:
!pip install torch transformers peft datasets bitsandbytes accelerate qwen-vl-utils -q

In [2]:
import os
import torch
import shutil
from google.colab import drive
from datasets import load_dataset
from transformers import (
    AutoModelForVision2Seq,
    AutoProcessor,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    default_data_collator
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# --- 1. AYARLAR ---
FILE_NAME = "data.jsonl"
MODEL_ID = "Qwen/Qwen2-VL-2B-Instruct"
OUTPUT_DIR = "/content/qwen2-vl-turkish-food-v2"

# Dosya kontrolü
if not os.path.exists(FILE_NAME):
    raise FileNotFoundError(f"❌ HATA: '{FILE_NAME}' yok! Lütfen soldan sürükleyip yükle.")

# --- 2. MODEL HAZIRLIĞI (T4 GPU UYUMLU) ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16 # T4 için float16 (bf16 değil!)
)

print("🚀 Model Yükleniyor...")
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID, quantization_config=bnb_config, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True
)
processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)

# LoRA Ayarları
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=64, lora_alpha=128,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

# --- 3. VERİ HAZIRLIĞI ---
def format_jsonl_to_chat(example):
    prompt = example['instruction'] + (f"\n{example['input']}" if example.get('input') else "")
    messages = [{"role": "user", "content": [{"type": "text", "text": prompt}]},
                {"role": "assistant", "content": [{"type": "text", "text": example['output']}]}]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    inputs = processor(text=[text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    inputs['labels'] = inputs['input_ids'].clone()
    inputs['labels'][inputs['labels'] == processor.tokenizer.pad_token_id] = -100
    return {"input_ids": inputs['input_ids'][0], "attention_mask": inputs['attention_mask'][0], "labels": inputs['labels'][0]}

print("📚 Veri işleniyor...")
dataset = load_dataset("json", data_files=FILE_NAME, split="train")
formatted_dataset = dataset.map(format_jsonl_to_chat, remove_columns=dataset.column_names)

# --- 4. EĞİTİM ---
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=10,    # 10 Epoch
    fp16=True,              # ✅ T4 GPU için AÇIK
    bf16=False,             # ❌ T4 için KAPALI
    logging_steps=5,
    save_strategy="epoch",
    optim="paged_adamw_8bit",
    report_to="none"
)

trainer = Trainer(model=model, args=training_args, train_dataset=formatted_dataset, data_collator=default_data_collator)

print("🔥 EĞİTİM BAŞLIYOR... (Hızlı bitecek)")
trainer.train()

# Kaydet
trainer.save_model(OUTPUT_DIR)
processor.save_pretrained(OUTPUT_DIR)
print("✅ Eğitim Bitti!")

# --- 5. DRIVE YEDEKLEME ---
print("📦 Google Drive'a yedekleniyor...")
drive.mount('/content/drive')
drive_dest = "/content/drive/My Drive/YapayZekaModellerim/TurkishFood-Qwen2B-Final"

if os.path.exists(drive_dest):
    shutil.rmtree(drive_dest)
shutil.copytree(OUTPUT_DIR, drive_dest)

print(f"🎉 TEBRİKLER! Modelin Drive'da: {drive_dest}")

🚀 Model Yükleniyor...


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

🔥 EĞİTİM BAŞLIYOR... (Hızlı bitecek)


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
5,3.174500
10,2.832600
15,2.093300
20,1.663100
25,1.723700
30,1.173700
35,0.925400
40,0.962400
45,0.534100
50,0.537100


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/pyt

✅ Eğitim Bitti!
📦 Google Drive'a yedekleniyor...
Mounted at /content/drive
🎉 TEBRİKLER! Modelin Drive'da: /content/drive/My Drive/YapayZekaModellerim/TurkishFood-Qwen2B-Final


In [ ]:
print("🧹 Eski ve bozuk sürümler temizleniyor...")
!pip uninstall -y bitsandbytes transformers peft accelerate

print("✨ En güncel ve uyumlu sürümler GitHub'dan yükleniyor...")
!pip install -U bitsandbytes
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U git+https://github.com/huggingface/peft.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install -U qwen-vl-utils

print("✅ Kurulum tamamlandı! Şimdi eğitim kodunu çalıştırabilirsin.")

🧹 Eski ve bozuk sürümler temizleniyor...
Found existing installation: bitsandbytes 0.48.2
Uninstalling bitsandbytes-0.48.2:
  Successfully uninstalled bitsandbytes-0.48.2
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
✨ En güncel ve uyumlu sürümler GitHub'dan yükleniyor...
  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl (59.4 MB)
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-91i5cxgn
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-91i5cxgn
  Resolved https://github.co

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-l0fbvdvh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-l0fbvdvh
  Resolved https://github.com/huggingface/peft.git to commit b10527e82c2171568f538f5b822817e8a753672a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)
  Created wheel for peft: filename=peft-0.18.1.dev0-py3-none-any.whl size=566748 sha256=e6dcf63b4bcd6c4bdaa6e7fcd507c2e2c85d014ff39ddb47c0484024fd035e1b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3v4csasi/wheels/5d/16/61/117d50be36b7cb532817817523554825ff840d223c0f65c2c4
Successfully built peft


  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-w9qx_kq4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-w9qx_kq4
  Resolved https://github.com/huggingface/accelerate.git to commit a72834ca8996f882208d4946d9180e626ae5c9c8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.13.0.dev0-py3-none-any.whl size=381249 sha256=e9d170ea922a74174c8c6e924a8a3c20bc267741843c3a1753d99dffb4796e7b
  Stored in directory: /tmp/pip-ephem-wheel-cache-2glzthoy/wheels/5a/20/fb/1221fe933b56fe7ac69fd00159d9a1950bc8ced38198abc18f
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.12.0
    Uninstalling accelerate-1.12.0:
      Successfully uninstalled accelerate-1.12.0


✅ Kurulum tamamlandı! Şimdi eğitim kodunu çalıştırabilirsin.


In [1]:
!pip install -U --force-reinstall bitsandbytes torch transformers peft datasets accelerate qwen-vl-utils -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from peft import PeftModel

# --- AYARLAR ---
BASE_MODEL_ID = "Qwen/Qwen2-VL-2B-Instruct"
ADAPTER_DIR = "./qwen2-vl-turkish-food-text-only"

print("⏳ Ana model yükleniyor...")
model = AutoModelForVision2Seq.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

processor = AutoProcessor.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)

print(f"📂 Adaptörler yükleniyor...")
try:
    model = PeftModel.from_pretrained(model, ADAPTER_DIR)
    print("✅ Hazır!")
except Exception as e:
    print(f"❌ Hata: {e}")

model.eval()

# --- GÜNCELLENMİŞ SORU SORMA FONKSİYONU ---
def modele_sor(soru):
    messages = [
        {"role": "user", "content": [{"type": "text", "text": soru}]}
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = processor(
        text=[text],
        padding=True,
        return_tensors="pt",
    ).to(model.device)

    # --- TEKRAR SORUNUNU ÇÖZEN AYARLAR BURADA ---
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2, # <-- YENİ: Tekrarı cezalandırır (1.0 ile 1.5 arası)
            no_repeat_ngram_size=3, # <-- YENİ: 3 kelimelik öbeklerin aynısını tekrarlamaz
            eos_token_id=processor.tokenizer.eos_token_id, # Cümle bitişini garantiye al
            pad_token_id=processor.tokenizer.pad_token_id
        )

    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    return output_text[0]

# --- HATA YÖNETİMİ OLAN DÖNGÜ ---
print("\n" + "="*40)
print("🤖 TÜRK MUTFAĞI ASİSTANI (Düzeltilmiş Versiyon)")
print("Çıkmak için 'Stop' butonuna basabilirsin.")
print("="*40 + "\n")

try:
    while True:
        soru = input("SORU SOR (Çıkış için q): ")
        if soru.lower() in ["q", "exit", "çık"]:
            print("Güle güle! 👋")
            break

        if soru.strip() == "":
            continue

        print("🤔 Düşünüyor...")
        cevap = modele_sor(soru)
        print(f"\nCEVAP:\n{cevap}\n")
        print("-" * 30)

except KeyboardInterrupt:
    # Stop butonuna basınca burası çalışır
    print("\n\n⛔ İşlem kullanıcı tarafından durduruldu. Güle güle! 👋")

⏳ Ana model yükleniyor...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

📂 Adaptörler yükleniyor...
❌ Hata: Can't find 'adapter_config.json' at './qwen2-vl-turkish-food-text-only'

🤖 TÜRK MUTFAĞI ASİSTANI (Düzeltilmiş Versiyon)
Çıkmak için 'Stop' butonuna basabilirsin.

SORU SOR (Çıkış için q): Geleneksel 'Şalgam Suyu' üretiminde kullanılan 'Setik' nedir? 
🤔 Düşünüyor...

CEVAP:
"Setik", genellikle "şalgam suyu" veya "sulama suyu"sürgünlerinin bir parçasıdır ve bu, suyunun sularını belirli bir şekilde düzenlemesine yardımcı olur. Bu yüzden, şalgam süyünün üretimi sırasında kullanılır.

Bu nedenle, Setik, şalga suyu'nun kirlenmesini önleyerek, suya daha iyi çözünürlük sağlar. Ayrıca, setik, suyu daha hızlı ve daha net bir şekilde sıvıda tutar.

------------------------------
SORU SOR (Çıkış için q): Geleneksel Türk mutfağında israfı önlemeye yönelik 'Tirit' gibi yemeklerin mantığı nedir?"
🤔 Düşünüyor...

CEVAP:
"Tirit" genellikle Türkiye'deki gelenekel bir yemeği ve içecekleri için kullanılan terimdir. Bu terimin anlamı, bu tür yemekteki "tirit", ya da diğer

In [ ]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from peft import PeftModel

# --- AYARLAR ---
BASE_MODEL_ID = "Qwen/Qwen2-VL-2B-Instruct"
ADAPTER_DIR = "./qwen2-vl-turkish-food-v2" #eğitim çıktısı

print(" Ana model yükleniyor...")
# 1. Ana Modeli Yükle
base_model = AutoModelForVision2Seq.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# 2. Processor Yükle
processor = AutoProcessor.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)

# 3. Senin Eğittiğin Adaptörü Yükle
print(f" Veriler ({ADAPTER_DIR}) modele ekleniyor...")
try:
    model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
    print(" Adaptör başarıyla yüklendi! Teste hazırız.")
except Exception as e:
    print(f" HATA: Adaptör bulunamadı. Klasör ismini kontrol et. Hata: {e}")
    # Hata olursa base modelle devam etsin
    model = base_model

model.eval()

# --- SORU CEVAP FONKSİYONU ---
def sor_bakalim(soru):
    messages = [
        {"role": "user", "content": [{"type": "text", "text": soru}]}
    ]

    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = processor(
        text=[text],
        padding=True,
        return_tensors="pt",
    ).to(model.device)

    # Parametreleri, modelin tekrara düşmemesi için ayarladık
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=200,    # Cevap uzunluğu
            do_sample=True,        # Yaratıcılık açık
            temperature=0.6,       # 0.6 biraz daha tutarlı yapar (0.1 robot, 1.0 şair)
            top_p=0.9,
            repetition_penalty=1.2, # Tekrarı önler
            no_repeat_ngram_size=3  # "Ali topu attı, Ali topu attı" gibi tekrarları engeller
        )

    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]

    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    return output_text[0]

# --- CANLI TEST PANELİ ---
print("\n" + "="*50)
print(" TEST AŞAMASI BAŞLADI")
print("Çıkmak için 'q' yazabilirsin.")
print("="*50 + "\n")

try:
    while True:
        soru = input("\nSen: ")
        if soru.lower() in ["q", "exit", "çık", "dur"]:
            print("Test bitti. Sonuçlar tatmin ediciyse merge işlemine geçebilirsin. 👋")
            break

        if len(soru) < 2: continue

        print(" Model Cevabı:")
        cevap = sor_bakalim(soru)
        print(f"👉 {cevap}")
        print("-" * 30)

except KeyboardInterrupt:
    print("\n Test kullanıcı tarafından durduruldu.")

Unrecognized keys in `rope_parameters` for 'rope_type'='default': {'mrope_section'}


 Ana model yükleniyor...


Loading weights:   0%|          | 0/729 [00:00<?, ?it/s]

 Veriler (./qwen2-vl-turkish-food-v2) modele ekleniyor...
 Adaptör başarıyla yüklendi! Teste hazırız.

 TEST AŞAMASI BAŞLADI
Çıkmak için 'q' yazabilirsin.


Sen: Geleneksel 'Şalgam Suyu' üretiminde kullanılan 'Setik' nedir?
 Model Cevabı:
👉 Şalgám suyu üretiminde ham madde olarak kullanılan bulgur ununa 'setik' adı verilir. Bu un, ekşi hamur mayası ile karıştırılarak ilk fermantasyon aşamasını başlatır.
------------------------------

Sen: Pekmezde HMF (Hidroksimetil Furfural) oluşumu neden risklidir?
 Model Cevabı:
👉 Pekiş mutfak kültüründe uzun süre kaynatılarak tüketilmemesi (pekmez tarhanası), hmf bileşikleri içerdiği için kan şekerini hızlı yükseltebilir ve diabet hastalarının beslenmesinde gerekli kiểmollere maruz bırakılır.
------------------------------

Sen: Maraş Tarhanası'nın diğer tarhanalardan farkı nedir?
 Model Cevabı:
👉 MARAŞ TARHANASI, YEREL Geleneksel tarhana yapımında kullanılan unun kendi akı ile kurutulmuş ve kapağı kapalı olarak üretilmesi nedeniyle darı ve kalsiy

In [1]:
print("🧹 Çakışan kütüphaneler siliniyor...")
!pip uninstall -y transformers accelerate peft bitsandbytes

print("✨ Uyumlu versiyonlar sıfırdan kuruluyor...")
!pip install -U transformers accelerate peft bitsandbytes datasets torch qwen-vl-utils

print("✅ Kurulum bitti! Lütfen aşağıyı oku 👇")

🧹 Çakışan kütüphaneler siliniyor...
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: bitsandbytes 0.48.2
Uninstalling bitsandbytes-0.48.2:
  Successfully uninstalled bitsandbytes-0.48.2
✨ Uyumlu versiyonlar sıfırdan kuruluyor...
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.18.0-py3-none-any.whl.metadata (14 kB)
  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)
Using cached peft-0.18.0-py3-n